In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
from fuzzywuzzy import fuzz
from datetime import date
import datetime as dt
import pandas as pd
import numpy as np
import random
from selenium.webdriver.chrome.options import Options
import re
from tqdm import tqdm
import warnings
import selenium
warnings.filterwarnings("ignore")

C:\Users\Pedro\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [8]:
driver = webdriver.Chrome()
driver.get("https://www.rotowire.com/daily/nba/optimizer.php")

In [22]:
df = pd.read_html(driver.page_source)[0]

In [28]:
df_2 = pd.read_html(driver.page_source)[0]

In [31]:
pd.concat([df,df_2]).drop_duplicates().to_csv("nba_salaries_2023_11_23.csv",index=False)

In [4]:
driver = webdriver.Chrome()
driver.get("https://betway.com/pt/sports/grp/soccer/england/premier-league")
links = pd.Series([val.get_attribute("href") for val in driver.find_elements("xpath","//a[@href]")])
links_text = [val.text for val in driver.find_elements("xpath","//a[@href]")]
links_df = pd.DataFrame(zip(links_text,links),columns=["GAME","URL"])
game_urls_df = links_df[links_df["URL"].str.contains("evt")]

In [5]:
full_odds_df = pd.DataFrame()
full_goal_odds_df = pd.DataFrame()
full_goal_player_odds_df = pd.DataFrame()

for i,row in tqdm(game_urls_df[:12].iterrows()):


    driver.get(row["URL"])

    time.sleep(5)

    elems_selector = driver.find_elements("class name","contentSelectorItemLabel")
    elems_selector_text = [val.text for val in elems_selector]
    elems_selector_df = pd.DataFrame(zip(elems_selector,elems_selector_text),columns=["ELEMS","TEXT"])
    
    if len(elems_selector_df)==0:
    
        driver.get(row["URL"])

        time.sleep(5)    
        elems_selector = driver.find_elements("class name","contentSelectorItemLabel")
        elems_selector_text = [val.text for val in elems_selector]
        elems_selector_df = pd.DataFrame(zip(elems_selector,elems_selector_text),columns=["ELEMS","TEXT"])  
        
    if len(elems_selector_df[elems_selector_df["TEXT"].str.contains("Jogadores")])>0:
        
        elems_selector_df[elems_selector_df["TEXT"].str.contains("Jogadores")]["ELEMS"].values[0].click()

        time.sleep(2)

        
        elems_itemheader_text = [val.text for val in driver.find_elements("class name","outcomeItemHeader")]
        odds_text = [val.text for val in driver.find_elements("class name","odds") if "Decimal" not in val.text]
        odds_df = pd.DataFrame(zip(elems_itemheader_text,odds_text),columns=["TEXT","ODDS"])

        odds_df = odds_df[~odds_df["TEXT"].str.contains(" e ")]
        odds_df = odds_df[~odds_df["TEXT"].str.contains("dará assistência para")]
        
        
        odds_df["TEXT"] = odds_df["TEXT"].apply(lambda x: re.sub("Para dar pelo menos uma","1+",x))
        odds_df = odds_df[odds_df["TEXT"].str.contains("\+")]
        
        odds_df["STAT"] = odds_df["TEXT"].apply(lambda x: x.split("+")[-1][1:])

        odds_df["NUMBER"] = odds_df["TEXT"].apply(lambda x: int(x.split("+")[0][-1]))
        odds_df["PLAYER"] = odds_df["TEXT"].apply(lambda x: x.split("+")[0][:-2])
        odds_df["ODDS"] = odds_df["ODDS"].apply(lambda x: re.sub(",",".",x)).astype(float)
        odds_df["GAME"] = row["GAME"]
        
        full_odds_df = pd.concat([full_odds_df,odds_df])
        
    if len(elems_selector_df[elems_selector_df["TEXT"].str.contains("Gols")])>0:

        elems_selector_df[elems_selector_df["TEXT"].str.contains("Gols")]["ELEMS"].values[0].click()

        time.sleep(2)

        collapse_series = pd.Series([val.text for val in driver.find_elements("class name","collapsablePanel")])

        collapse_series = collapse_series[collapse_series.str.contains("^Equipe")]

        title_list = collapse_series.apply(lambda x: x.split("\n")[0])
        mais_de_list = collapse_series.apply(lambda x: x.split("\n")[4])
        menos_de_list = collapse_series.apply(lambda x: x.split("\n")[5])    

        goal_odds_df = pd.DataFrame(zip(title_list,mais_de_list,menos_de_list),columns=["TEXT","ODDS_MAIS","ODDS_MENOS"])

        goal_odds_df["NUMBER"] = goal_odds_df["TEXT"].apply(lambda x: x[-3:])
        goal_odds_df["ODDS_MAIS"] = goal_odds_df["ODDS_MAIS"].apply(lambda x: re.sub(",",".",x)).astype(float)
        goal_odds_df["ODDS_MENOS"] = goal_odds_df["ODDS_MENOS"].apply(lambda x: re.sub(",",".",x)).astype(float)    
        goal_odds_df["GAME"] = row["GAME"]
        goal_odds_df["EQUIPE"] = ""

        goal_odds_df.loc[goal_odds_df["TEXT"].str.contains("Equipe A"),"EQUIPE"] = row["GAME"].split(" -")[0]
        goal_odds_df.loc[goal_odds_df["TEXT"].str.contains("Equipe B"),"EQUIPE"] = row["GAME"].split(" -")[1]

        full_goal_odds_df = pd.concat([full_goal_odds_df,goal_odds_df])       
        
    if len(elems_selector_df[elems_selector_df["TEXT"].str.contains("Artilh")])>0:
        
        elems_selector_df[elems_selector_df["TEXT"].str.contains("Artilh")]["ELEMS"].values[0].click()

        time.sleep(2)
        if len(driver.find_elements("class name","cookiePolicyAcceptButton"))>0:
            driver.find_elements("class name","cookiePolicyAcceptButton")[0].click()   
        time.sleep(1)
        
        for elem in driver.find_elements("class name","collapsableHeader"):
            try:
                elem.click()
            except:
                body = driver.find_element("css selector",'body')
                body.send_keys(Keys.PAGE_DOWN)
                elem.click()

        elems = driver.find_elements("class name","collapsableHeader")
        elems_text = [val.text for val in elems]
        collapse_df = pd.DataFrame(zip(elems,elems_text),columns=["ELEMS","TEXT"])
        collapse_df = collapse_df[collapse_df["TEXT"].str.contains("^Jogador para marcar exatamente")]


        for i,row_col in collapse_df.iterrows():



            row_col["ELEMS"].click()

            dropdown_elems = driver.find_elements("class name","dropdownSelectedOption")
            dropdown_text = [val.text for val in dropdown_elems]
            dropdown_df = pd.DataFrame(zip(dropdown_elems,dropdown_text),columns=["ELEMS","TEXT"])

            dropdown_df = dropdown_df[~dropdown_df["ELEMS"].isin(["Decimal","Ajuda","Português"])]
            dropdown_df["ELEMS"].values[-1].click()

            dropdown_series = pd.Series([val.text for val in driver.find_elements("class name","dropdownItem")])

            dropdown_series = dropdown_series[dropdown_series.str.contains("\,")]

            player_series = dropdown_series.apply(lambda x: x.split("\n")[0])
            player_odds = dropdown_series.apply(lambda x: x.split("\n")[1])
            local_df = pd.DataFrame(zip(player_series,player_odds),columns=["PLAYER","ODDS"])    

            row_col["ELEMS"].click()    
            local_df["TEXT"] = row_col["TEXT"]
            local_df["GAME"] = row["GAME"]

            full_goal_player_odds_df = pd.concat([full_goal_player_odds_df,local_df])        

12it [06:46, 33.91s/it]


In [6]:
full_odds_df["PROBA"] = 1/full_odds_df["ODDS"]

In [7]:
full_goal_odds_df["PROBA_MAIS"] = 1/full_goal_odds_df["ODDS_MAIS"]
full_goal_odds_df["PROBA_MENOS"] = 1/full_goal_odds_df["ODDS_MENOS"]

In [8]:
full_goal_player_odds_df["ODDS"] = full_goal_player_odds_df["ODDS"].apply(lambda x: re.sub(",",".",x)).astype(float)

In [9]:
full_goal_player_odds_df["NUMBER"] = full_goal_player_odds_df["TEXT"].apply(lambda x: re.findall("\d",x)[0])
full_goal_player_odds_df["PROBA"] = 1/full_goal_player_odds_df["ODDS"]

In [10]:
full_odds_df.to_csv("premier_league_player_odds.csv",index=False)
full_goal_odds_df.to_csv("premier_league_team_goal_odds.csv",index=False)
full_goal_player_odds_df.to_csv("premier_league_player_goal_odds.csv",index=False)

In [11]:
full_odds_df

,TEXT,ODDS,STAT,NUMBER,PLAYER,GAME,PROBA
0,Odsonne Edouard 1+ Finalizações no gol,1.25,Finalizações no gol,1,Odsonne Edouard,Crystal Palace -Fulham,0.800000
1,Odsonne Edouard 2+ Finalizações no gol,2.50,Finalizações no gol,2,Odsonne Edouard,Crystal Palace -Fulham,0.400000
2,Odsonne Edouard 3+ Finalizações no gol,6.00,Finalizações no gol,3,Odsonne Edouard,Crystal Palace -Fulham,0.166667
3,Odsonne Edouard 4+ Finalizações no gol,13.00,Finalizações no gol,4,Odsonne Edouard,Crystal Palace -Fulham,0.076923
4,Eberechi Eze 1+ Finalizações no gol,1.61,Finalizações no gol,1,Eberechi Eze,Crystal Palace -Fulham,0.621118
...,...,...,...,...,...,...,...
426,Anthony Gordon 1+ Foras de jogo,3.60,Foras de jogo,1,Anthony Gordon,Sheffield United -Newcastle United,0.277778
427,Alexander Isak 1+ Foras de jogo,2.37,Foras de jogo,1,Alexander Isak,Sheffield United -Newcastle United,0.421941
428,Elliot Anderson 1+ Foras de jogo,4.33,Foras de jogo,1,Elliot Anderson,Sheffield United -Newcastle United,0.230947
429,Jacob Murphy 1+ Foras de jogo,3.60,Foras de jogo,1,Jacob Murphy,Sheffield United -Newcastle United,0.277778


In [12]:
full_goal_odds_df

,TEXT,ODDS_MAIS,ODDS_MENOS,NUMBER,GAME,EQUIPE,PROBA_MAIS,PROBA_MENOS
0,Equipe A Total de gols 0.5,1.22,3.800,0.5,Crystal Palace -Fulham,Crystal Palace,0.819672,0.263158
1,Equipe A Total de gols 1.5,2.00,1.720,1.5,Crystal Palace -Fulham,Crystal Palace,0.500000,0.581395
2,Equipe A Total de gols 2.5,4.00,1.200,2.5,Crystal Palace -Fulham,Crystal Palace,0.250000,0.833333
3,Equipe A Total de gols 3.5,9.00,1.030,3.5,Crystal Palace -Fulham,Crystal Palace,0.111111,0.970874
4,Equipe B Total de gols 0.5,1.47,2.500,0.5,Crystal Palace -Fulham,Fulham,0.680272,0.400000
...,...,...,...,...,...,...,...,...
4,Equipe B Total de gols 0.5,1.15,4.750,0.5,Aston Villa -Brighton,Brighton,0.869565,0.210526
5,Equipe B Total de gols 1.5,1.80,1.900,1.5,Aston Villa -Brighton,Brighton,0.555556,0.526316
6,Equipe B Total de gols 2.5,3.40,1.280,2.5,Aston Villa -Brighton,Brighton,0.294118,0.781250
7,Equipe B Total de gols 3.5,7.00,1.070,3.5,Aston Villa -Brighton,Brighton,0.142857,0.934579


In [13]:
full_goal_player_odds_df

,PLAYER,ODDS,TEXT,GAME,NUMBER,PROBA
0,Odsonne Edouard,2.80,Jogador para marcar exatamente 1 gol,Crystal Palace -Fulham,1,0.357143
1,Eberechi Eze,3.00,Jogador para marcar exatamente 1 gol,Crystal Palace -Fulham,1,0.333333
2,Jean-Philippe Mateta,3.10,Jogador para marcar exatamente 1 gol,Crystal Palace -Fulham,1,0.322581
3,Jesurun Rak-Sakyi,3.30,Jogador para marcar exatamente 1 gol,Crystal Palace -Fulham,1,0.303030
4,Jordan Ayew,3.75,Jogador para marcar exatamente 1 gol,Crystal Palace -Fulham,1,0.266667
...,...,...,...,...,...,...
5,Harvey Barnes,41.00,Jogador para marcar exatamente 3 gols,Sheffield United -Newcastle United,3,0.024390
6,Miguel Almirón,67.00,Jogador para marcar exatamente 3 gols,Sheffield United -Newcastle United,3,0.014925
7,Jacob Murphy,81.00,Jogador para marcar exatamente 3 gols,Sheffield United -Newcastle United,3,0.012346
8,Elliot Anderson,151.00,Jogador para marcar exatamente 3 gols,Sheffield United -Newcastle United,3,0.006623


In [14]:
full_goal_player_odds_df[full_goal_player_odds_df["PLAYER"].str.contains("Pedro")]

,PLAYER,ODDS,TEXT,GAME,NUMBER,PROBA
25,Pedro Neto,3.8,Jogador para marcar exatamente 1 gol,Luton Town -Wolverhampton,1,0.263158
18,Pedro Neto,13.0,Jogador para marcar exatamente 2 gols,Luton Town -Wolverhampton,2,0.076923
11,Pedro Neto,101.0,Jogador para marcar exatamente 3 gols,Luton Town -Wolverhampton,3,0.009901
26,Pedro Porro,7.0,Jogador para marcar exatamente 1 gol,Arsenal -Tottenham,1,0.142857
24,Pedro Porro,81.0,Jogador para marcar exatamente 2 gols,Arsenal -Tottenham,2,0.012346
3,João Pedro,2.5,Jogador para marcar exatamente 1 gol,Brighton -Bournemouth,1,0.400000
3,João Pedro,6.5,Jogador para marcar exatamente 2 gols,Brighton -Bournemouth,2,0.153846
3,João Pedro,19.0,Jogador para marcar exatamente 3 gols,Brighton -Bournemouth,3,0.052632


In [15]:
full_goal_player_odds_df[full_goal_player_odds_df["PLAYER"].str.contains("Martin")]

,PLAYER,ODDS,TEXT,GAME,NUMBER,PROBA
38,Lisandro Martinez,7.5,Jogador para marcar exatamente 1 gol,Burnley -Manchester Utd,1,0.133333
33,Lisandro Martinez,101.0,Jogador para marcar exatamente 2 gols,Burnley -Manchester Utd,2,0.009901
3,Gabriel Martinelli,3.1,Jogador para marcar exatamente 1 gol,Arsenal -Tottenham,1,0.322581
5,Martin Ødegaard,3.3,Jogador para marcar exatamente 1 gol,Arsenal -Tottenham,1,0.303030
3,Gabriel Martinelli,9.0,Jogador para marcar exatamente 2 gols,Arsenal -Tottenham,2,0.111111
6,Martin Ødegaard,11.0,Jogador para marcar exatamente 2 gols,Arsenal -Tottenham,2,0.090909
3,Gabriel Martinelli,41.0,Jogador para marcar exatamente 3 gols,Arsenal -Tottenham,3,0.024390
6,Martin Ødegaard,67.0,Jogador para marcar exatamente 3 gols,Arsenal -Tottenham,3,0.014925
